In [ ]:
# 0. Instalar librerías necesarias (solo la primera vez)
!pip install transformers datasets evaluate rouge-score accelerate -q

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 50.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 34.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 28.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 105.8 MB/s eta 0:00:00


In [ ]:
# 1. Montar Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# 2. Importaciones necesarias
import os, glob
import torch
import evaluate
from transformers import (
    BartTokenizer, BartForConditionalGeneration,
    Seq2SeqTrainer, Seq2SeqTrainingArguments,
    DataCollatorForSeq2Seq
)
from datasets import load_from_disk

# Cargar datasets preprocesados
base_path = "/content/drive/MyDrive/LLMPractica/preprocessedatasets"
train_dataset = load_from_disk(os.path.join(base_path, "train_dataset"))
val_dataset = load_from_disk(os.path.join(base_path, "val_dataset"))
test_dataset = load_from_disk(os.path.join(base_path, "test_dataset"))

# Checkpoints
checkpoint_dir = "/content/drive/MyDrive/LLMPractica/bart-json-output"
os.makedirs(checkpoint_dir, exist_ok=True)
checkpoint_paths = sorted(glob.glob(os.path.join(checkpoint_dir, "checkpoint-*")), reverse=True)
last_checkpoint = next((p for p in checkpoint_paths if os.path.isfile(os.path.join(p, "pytorch_model.bin"))), None)

# Cargar modelo y tokenizer
if last_checkpoint:
    print(f"🔄 Cargando modelo desde checkpoint válido: {last_checkpoint}")
    model = BartForConditionalGeneration.from_pretrained(last_checkpoint)
    tokenizer = BartTokenizer.from_pretrained(last_checkpoint)
else:
    print("🚀 No hay checkpoints válidos. Se carga el modelo base.")
    model = BartForConditionalGeneration.from_pretrained("facebook/bart-large-cnn")
    tokenizer = BartTokenizer.from_pretrained("facebook/bart-large-cnn")

# Métrica ROUGE
rouge = evaluate.load("rouge")
def compute_metrics(eval_pred):
    preds, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    return {k: round(v, 4) for k, v in rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True).items()}

# Configuración del entrenamiento
training_args = Seq2SeqTrainingArguments(
    output_dir=checkpoint_dir,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=3,
    learning_rate=2e-5,
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_dir="/content/drive/MyDrive/LLMPractica/logs",
    logging_steps=10,
    predict_with_generate=True,
    generation_max_length=128,
    fp16=torch.cuda.is_available(),
    report_to="none"
)

# Trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=DataCollatorForSeq2Seq(tokenizer, model=model),
    compute_metrics=compute_metrics
)

# Entrenar
trainer.train(resume_from_checkpoint=last_checkpoint)

# Evaluar
results = trainer.evaluate(test_dataset)
print("\n📊 Resultados ROUGE en test:", results)

# Prueba de resumen
def summarize(text):
    input_text = "Summary: " + text
    inputs = tokenizer(input_text, return_tensors="pt", max_length=1024, truncation=True)
    inputs = {k: v.to(model.device) for k, v in inputs.items()}
    output_ids = model.generate(**inputs, max_length=128, num_beams=4, no_repeat_ngram_size=3, repetition_penalty=1.2)
    return tokenizer.decode(output_ids[0], skip_special_tokens=True)


# Prueba con un texto del test y guardar el resumen
print("\n--- Ejemplo de prueba ---")
sample_text = test_dataset[10]["text"]
resumen_generado = summarize(sample_text)

print("Texto original:\n", sample_text)
print("Resumen generado:\n", resumen_generado)

# Guardar el resumen generado
output_dir = "/content/drive/MyDrive/LLMPractica/textfinetunned"
os.makedirs(output_dir, exist_ok=True)
output_path = os.path.join(output_dir, "resumen_generado.txt")

with open(output_path, "w", encoding="utf-8") as f:
    f.write(resumen_generado)

print(f"\n✅ Resumen guardado en: {output_path}")
torch.cuda.empty_cache()

🚀 No hay checkpoints válidos. Se carga el modelo base.


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum
1,0.981300,0.415570,0.821000,0.802700,0.816700,0.818600
2,0.249400,0.172731,0.818600,0.800900,0.816500,0.816900
3,0.127500,0.133958,0.824500,0.806300,0.821400,0.823500


/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:3464: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 142, 'min_length': 56, 'early_stopping': True, 'num_beams': 4, 'length_penalty': 2.0, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(



📊 Resultados ROUGE en test: {'eval_loss': 0.15179197490215302, 'eval_rouge1': 0.7381, 'eval_rouge2': 0.7122, 'eval_rougeL': 0.7396, 'eval_rougeLsum': 0.7385, 'eval_runtime': 38.1133, 'eval_samples_per_second': 0.525, 'eval_steps_per_second': 0.262, 'epoch': 3.0}

--- Ejemplo de prueba ---
Texto original:
 Through unwavering determination, Galileo Galilei became a symbol of resistance against oppression. Their actions led to major reforms and left an enduring legacy in the fight for freedom and human rights.
Resumen generado:
 Through unwavering determination, Galileo Galilei became a symbol of resistance against oppression. A symbolic scene captures the essence of this historical turning point. This moment could be captured in a vivid historical scene or historical scene. This could be the perfect scene for a historical reenactment.

✅ Resumen guardado en: /content/drive/MyDrive/LLMPractica/textfinetunned/resumen_generado.txt


In [ ]:
# Ejemplo aparte de QLoRA y cuantificación (para futuros proyectos)
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import LoraConfig, get_peft_model
import torch

# Cargar modelo base con cuantificación (4 bits)
model_name = "meta-llama/Llama-2-7b-hf"  # Cambia por tu modelo
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    load_in_4bit=True,  # Cuantificación 4 bits
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Configuración LoRA
lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

# Aplicar QLoRA
model = get_peft_model(model, lora_config)

# Ejemplo de inferencia
inputs = tokenizer("Ejemplo de texto", return_tensors="pt").to(model.device)
outputs = model.generate(**inputs, max_new_tokens=50)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))